In [2]:
from transformers import GPTNeoXForCausalLM, AutoTokenizer
from peft import PeftModel, LoraConfig, get_peft_model
from accelerate import Accelerator
import torch
import torch.nn as nn

import math

In [5]:
class args:
    rank = 8
    lora_alpha = 32
    target_modules = ["query_key_value"]


In [3]:

# Load the model
base_model = GPTNeoXForCausalLM.from_pretrained("EleutherAI/pythia-410m")


In [4]:
base_model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 1024)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
  

In [6]:
# Add lora adapter

lora_config = LoraConfig(
    r=args.rank,
    lora_alpha=args.lora_alpha,
    lora_dropout=0.05,
    target_modules=args.target_modules,
    bias="none",
    task_type="CAUSAL_LM"
)


In [7]:
lora_model = get_peft_model(base_model, lora_config)

In [8]:
lora_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(50304, 1024)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-23): 24 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=3072, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (defaul

In [20]:
operator.attrgetter(lora_model, "gpt_neox.layers")

AttributeError: 'GPTNeoXModel' object has no attribute 'layers.0'

In [18]:
lora_model.gpt_neox.layers

ModuleList(
  (0-23): 24 x GPTNeoXLayer(
    (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (post_attention_dropout): Dropout(p=0.0, inplace=False)
    (post_mlp_dropout): Dropout(p=0.0, inplace=False)
    (attention): GPTNeoXAttention(
      (rotary_emb): GPTNeoXRotaryEmbedding()
      (query_key_value): lora.Linear(
        (base_layer): Linear(in_features=1024, out_features=3072, bias=True)
        (lora_dropout): ModuleDict(
          (default): Dropout(p=0.05, inplace=False)
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=1024, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=3072, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
      )
      (dense): Linear(in_features=1024, out_features=1024, bias

In [17]:
getattr(lora_model, "gpt_neox.layers.0.attention.query_key_value.lora_A")

AttributeError: 'GPTNeoXForCausalLM' object has no attribute 'gpt_neox.layers'